In [5]:
# import torch
# print(torch.cuda.is_available())
# print(torch.cuda.get_device_name(0))

True
NVIDIA GeForce RTX 3060 Laptop GPU


In [6]:
# !pip install transformers datasets accelerate wandb

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/10.0 MB ? eta -:--:--
   -------------------- ------------------- 5.2/10.0 MB 31.9 MB/s eta 0:00:01
   ---------------------------------------- 10.0/10.0 MB 34.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/15.5 MB ? eta -:--:--
   ------------------------ --------------- 9.4/15.5 MB 45.2 MB/s eta 0:00:01
   ---------------------------------------- 15.5/15.5 MB 42.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
   ---------------------------------------- 2.4/2.4 MB 27.2 MB/s eta 0:00:00


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


__Get the dataset__

In [1]:
!nvidia-smi

Sat Nov 30 16:02:26 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 556.12                 Driver Version: 556.12         CUDA Version: 12.5     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3060 ...  WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   35C    P0             25W /  115W |       0MiB /   6144MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
# specify gpu for torch
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [3]:
from datasets import load_dataset

dataset = load_dataset("cnn_dailymail", '3.0.0')

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})

In [5]:
import re
def clean_text(text):
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'\[[^\]]*\]', '', text)
    return text
dataset = dataset.map(lambda x: {'article': clean_text(x['article'])})

In [7]:
# Split the dataset into **training** and **validation** sets.

train_test_data = dataset['train'].train_test_split(test_size=0.1)
train_data = train_test_data['train']
val_data = train_test_data['test']


In [8]:
from transformers import AutoTokenizer

# tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")

tokenizer = AutoTokenizer.from_pretrained("google-t5/t5-small")


# def tokenize_function(examples):
#   return tokenizer(examples["article"], max_length=1024, truncation=True)

#tokenized_data = train_data.map(tokenize_function, batched=True)
# Assuming `tokenizer` is initialized

# max_length=1024
max_length=512

train_data = train_data.map(lambda x: tokenizer(x['article'], padding='max_length', truncation=True, max_length=max_length), batched=True)
val_data = val_data.map(lambda x: tokenizer(x['article'], padding='max_length', truncation=True, max_length=max_length), batched=True)

# Set the labels for the model (if needed, e.g., for summarization)
train_data = train_data.map(lambda x: {'labels': tokenizer(x['highlights'], padding='max_length', truncation=True, max_length=max_length)['input_ids']})
val_data = val_data.map(lambda x: {'labels': tokenizer(x['highlights'], padding='max_length', truncation=True, max_length=max_length)['input_ids']})


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

C:\Users\Alex\AppData\Roaming\Python\Python312\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Alex\.cache\huggingface\hub\models--google-t5--t5-small. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Map:   0%|          | 0/258401 [00:00<?, ? examples/s]

Map:   0%|          | 0/28712 [00:00<?, ? examples/s]

Map:   0%|          | 0/258401 [00:00<?, ? examples/s]

Map:   0%|          | 0/28712 [00:00<?, ? examples/s]

In [10]:
train_data

Dataset({
    features: ['article', 'highlights', 'id', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 258401
})

In [17]:
import pickle 

my_object_512 = {'trainData': train_data, 'valData': val_data}

with open('my_object_512.pkl', 'wb') as f:
    pickle.dump(my_object_512, f)

In [11]:
#Choose a Pre-trained Model - Use a pre-trained model like `BART`, `T5`, or `Pegasus` for summarization: pretrain on datasets, last layer retrain,

from transformers import AutoModelForSeq2SeqLM

# model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large-cnn")

# smaller model
model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")



In [12]:
# Training the Model: Fine-tune the model using your dataset. Hugging Face’s 'Trainer' API simplifies the process:

from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01
    )

# training_args = TrainingArguments(
#     output_dir="./results/t5-small",  # Save directory
#     evaluation_strategy="epoch",     # Evaluate at each epoch
#     learning_rate=5e-5,              # Adjusted learning rate
#     per_device_train_batch_size=8,   # Batch size per device
#     per_device_eval_batch_size=8,
#     num_train_epochs=3,              # Number of epochs
#     weight_decay=0.01,               # Regularization
#     save_strategy="epoch",           # Save checkpoint at each epoch
#     save_total_limit=2,              # Keep only 2 checkpoints
#     logging_dir="./logs",            # Log directory
#     logging_steps=200,               # Log every 200 steps
# )

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=val_data,
    )

trainer.train()


C:\Users\Alex\AppData\Roaming\Python\Python312\site-packages\transformers\training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: agscheue (agscheue-umich). Use `wandb login --relogin` to force relogin


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,0.291600,0.266130
2,0.296600,0.264532
3,0.296700,0.264025


TrainOutput(global_step=193803, training_loss=0.2981521848865768, metrics={'train_runtime': 63798.7488, 'train_samples_per_second': 12.151, 'train_steps_per_second': 3.038, 'total_flos': 1.0491737052649882e+17, 'train_loss': 0.2981521848865768, 'epoch': 3.0})

In [13]:
## Save model
model.save_pretrained("./T5-ai-model")
# Save the tokenizer
tokenizer.save_pretrained('./T5-ai-model')

('./T5-ai-model\\tokenizer_config.json',
 './T5-ai-model\\special_tokens_map.json',
 './T5-ai-model\\tokenizer.json')